# Случайный лес (Random Forest)

## Введение

Random Forest — это ансамблевый алгоритм, который строит множество решающих деревьев и объединяет их предсказания. Это один из самых популярных и эффективных алгоритмов машинного обучения.

### Применение в биологии:
- Классификация типов опухолей по экспрессии генов
- Предсказание взаимодействий белок-белок
- Анализ важности генов для заболеваний
- Предсказание структуры белков
- Диагностика заболеваний по множеству биомаркеров

### Ключевая идея:
**Решающие деревья имеют высокий variance → Random Forest снижает variance через усреднение множества деревьев!**

### Основные принципы:
1. **Bagging (Bootstrap Aggregating)**: Каждое дерево обучается на случайной подвыборке данных (с возвращением)
2. **Random Subspace Method**: Для каждого разбиения используется случайное подмножество признаков
3. **Усреднение предсказаний**: Финальное предсказание = среднее/голосование всех деревьев

$$\text{Variance}_{\text{RF}} \approx \frac{\text{Variance}_{\text{tree}}}{n_{\text{trees}}}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.datasets import make_moons, make_classification
import pandas as pd

np.random.seed(42)

## 1. Сравнение одиночного дерева и Random Forest

### Демонстрация снижения variance

In [ ]:
# Генерируем данные
def true_function(x):
    return np.sin(x) * x + 0.5 * x

n_samples = 100
X_train = np.sort(np.random.uniform(0, 10, n_samples))
y_train = true_function(X_train) + np.random.normal(0, 0.5, n_samples)

X_test = np.linspace(0, 10, 200)
y_test_true = true_function(X_test)

X_train_2d = X_train.reshape(-1, 1)
X_test_2d = X_test.reshape(-1, 1)

print(f"Размер выборки: {n_samples}")

In [ ]:
# Обучаем модели
single_tree = DecisionTreeRegressor(max_depth=10, random_state=42)
single_tree.fit(X_train_2d, y_train)

random_forest = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
random_forest.fit(X_train_2d, y_train)

# Предсказания
y_pred_tree = single_tree.predict(X_test_2d)
y_pred_rf = random_forest.predict(X_test_2d)

# MSE на тренировочной выборке
train_mse_tree = mean_squared_error(y_train, single_tree.predict(X_train_2d))
train_mse_rf = mean_squared_error(y_train, random_forest.predict(X_train_2d))

print(f"\nОдиночное дерево - Train MSE: {train_mse_tree:.3f}")
print(f"Random Forest    - Train MSE: {train_mse_rf:.3f}")

In [ ]:
# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Одиночное дерево
axes[0].scatter(X_train, y_train, alpha=0.4, s=30, label='Данные')
axes[0].plot(X_test, y_test_true, 'g--', lw=2, label='Истинная функция')
axes[0].plot(X_test, y_pred_tree, 'r-', lw=2, label='Предсказание (1 дерево)')
axes[0].set_xlabel('X')
axes[0].set_ylabel('y')
axes[0].set_title('Одиночное дерево (max_depth=10)\nВысокий VARIANCE', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Random Forest
axes[1].scatter(X_train, y_train, alpha=0.4, s=30, label='Данные')
axes[1].plot(X_test, y_test_true, 'g--', lw=2, label='Истинная функция')
axes[1].plot(X_test, y_pred_rf, 'b-', lw=2, label='Предсказание (100 деревьев)')
axes[1].set_xlabel('X')
axes[1].set_ylabel('y')
axes[1].set_title('Random Forest (n_estimators=100)\nСниженный VARIANCE', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Random Forest дает более гладкие предсказания за счет усреднения!")

## 2. Bias-Variance Tradeoff для Random Forest

### Теория:

Для Random Forest важны два основных гиперпараметра:

1. **n_estimators** (количество деревьев):
   - Больше деревьев → меньше variance
   - Bias не меняется
   - **Почти никогда не приводит к overfitting!**

2. **max_depth** (глубина деревьев):
   - Больше глубина → меньше bias, больше variance
   - Аналогично одиночному дереву, но variance ниже

### Математика:

Для независимых деревьев:
$$\text{Var}(\text{среднее}) = \frac{\sigma^2}{n}$$

Для коррелированных деревьев:
$$\text{Var}(\text{RF}) = \rho\sigma^2 + \frac{1-\rho}{n}\sigma^2$$

где $\rho$ — корреляция между деревьями, $n$ — количество деревьев

**Random subspace method снижает корреляцию между деревьями!**

### Эксперимент: Влияние количества деревьев

In [ ]:
# Функция для вычисления bias и variance
def compute_bias_variance_rf(n_estimators, max_depth=None, n_iterations=50):
    """Вычисляет bias и variance для Random Forest"""
    predictions = np.zeros((n_iterations, len(X_test)))
    
    for i in range(n_iterations):
        # Генерируем новую выборку
        X_sample = np.sort(np.random.uniform(0, 10, n_samples))
        y_sample = true_function(X_sample) + np.random.normal(0, 0.5, n_samples)
        
        # Обучаем Random Forest
        rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, 
                                   random_state=i, n_jobs=-1)
        rf.fit(X_sample.reshape(-1, 1), y_sample)
        
        # Предсказания
        predictions[i] = rf.predict(X_test_2d)
    
    # Среднее предсказание
    mean_prediction = np.mean(predictions, axis=0)
    
    # Bias^2 и Variance
    bias_squared = np.mean((mean_prediction - y_test_true)**2)
    variance = np.mean(np.var(predictions, axis=0))
    
    return bias_squared, variance

# Эксперимент: влияние n_estimators
n_trees_range = [1, 5, 10, 20, 50, 100, 200]
biases_trees = []
variances_trees = []

print("Вычисление bias и variance для разного количества деревьев...")
for n_trees in n_trees_range:
    bias_sq, var = compute_bias_variance_rf(n_trees, max_depth=10, n_iterations=30)
    biases_trees.append(bias_sq)
    variances_trees.append(var)
    print(f"n_estimators={n_trees:3d}: Bias² = {bias_sq:.3f}, Variance = {var:.3f}")

print("\nГотово!")

In [ ]:
# График зависимости от количества деревьев
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(n_trees_range, biases_trees, 'b-o', lw=2, markersize=8, label='Bias²')
ax.plot(n_trees_range, variances_trees, 'r-s', lw=2, markersize=8, label='Variance')
total_error = np.array(biases_trees) + np.array(variances_trees)
ax.plot(n_trees_range, total_error, 'g-^', lw=2.5, markersize=8, label='Total Error')

ax.set_xlabel('Количество деревьев (n_estimators)', fontsize=12)
ax.set_ylabel('Ошибка', fontsize=12)
ax.set_title('Влияние количества деревьев на Bias-Variance\n(max_depth=10)', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xscale('log')

# Аннотация
ax.annotate('Bias НЕ меняется!', xy=(100, biases_trees[-1]), xytext=(30, biases_trees[-1]+0.5),
            arrowprops=dict(arrowstyle='->', color='blue', lw=1.5),
            fontsize=11, color='blue', fontweight='bold')

ax.annotate('Variance уменьшается', xy=(100, variances_trees[-1]), xytext=(20, variances_trees[-1]-1),
            arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
            fontsize=11, color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Увеличение количества деревьев снижает variance, не влияя на bias!")
print("✓ После ~50-100 деревьев улучшение замедляется")

### Эксперимент: Влияние глубины деревьев

In [ ]:
# Эксперимент: влияние max_depth
depths_range = range(1, 16)
biases_depth = []
variances_depth = []

print("Вычисление bias и variance для разной глубины деревьев...")
for depth in depths_range:
    bias_sq, var = compute_bias_variance_rf(n_estimators=100, max_depth=depth, n_iterations=30)
    biases_depth.append(bias_sq)
    variances_depth.append(var)
    print(f"max_depth={depth:2d}: Bias² = {bias_sq:.3f}, Variance = {var:.3f}")

print("\nГотово!")

In [ ]:
# График зависимости от глубины
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(depths_range, biases_depth, 'b-o', lw=2, markersize=6, label='Bias²')
ax.plot(depths_range, variances_depth, 'r-s', lw=2, markersize=6, label='Variance')
total_error_depth = np.array(biases_depth) + np.array(variances_depth)
ax.plot(depths_range, total_error_depth, 'g-^', lw=2.5, markersize=6, label='Total Error')

optimal_depth_rf = list(depths_range)[np.argmin(total_error_depth)]
ax.axvline(optimal_depth_rf, color='purple', linestyle='--', lw=2, alpha=0.7,
          label=f'Оптимальная глубина ≈ {optimal_depth_rf}')

ax.set_xlabel('Максимальная глубина деревьев (max_depth)', fontsize=12)
ax.set_ylabel('Ошибка', fontsize=12)
ax.set_title('Влияние глубины деревьев на Bias-Variance\n(n_estimators=100)', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ Оптимальная глубина для Random Forest: {optimal_depth_rf}")
print(f"  Variance Random Forest ниже, чем у одиночного дерева той же глубины!")

## 3. Классификация: Диагностика заболевания

Используем Random Forest для классификации пациентов.

In [ ]:
# Генерируем данные
X_class, y_class = make_moons(n_samples=500, noise=0.3, random_state=42)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_class, y_class, test_size=0.3, random_state=42
)

print(f"Тренировочная выборка: {len(X_train_c)} пациентов")
print(f"Тестовая выборка: {len(X_test_c)} пациентов")

In [ ]:
# Сравнение одиночного дерева и Random Forest
tree_clf = DecisionTreeClassifier(max_depth=10, random_state=42)
tree_clf.fit(X_train_c, y_train_c)

rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_clf.fit(X_train_c, y_train_c)

# Оценка
tree_train_acc = accuracy_score(y_train_c, tree_clf.predict(X_train_c))
tree_test_acc = accuracy_score(y_test_c, tree_clf.predict(X_test_c))

rf_train_acc = accuracy_score(y_train_c, rf_clf.predict(X_train_c))
rf_test_acc = accuracy_score(y_test_c, rf_clf.predict(X_test_c))

print("\n" + "="*50)
print("СРАВНЕНИЕ МОДЕЛЕЙ")
print("="*50)
print(f"Одиночное дерево:")
print(f"  Train Accuracy: {tree_train_acc:.2%}")
print(f"  Test Accuracy:  {tree_test_acc:.2%}")
print(f"  Overfitting gap: {(tree_train_acc - tree_test_acc):.2%}")
print(f"\nRandom Forest:")
print(f"  Train Accuracy: {rf_train_acc:.2%}")
print(f"  Test Accuracy:  {rf_test_acc:.2%}")
print(f"  Overfitting gap: {(rf_train_acc - rf_test_acc):.2%}")
print(f"\n✓ Random Forest имеет меньший overfitting gap!")

### Визуализация границ решений

In [ ]:
def plot_decision_boundary(model, X, y, ax, title):
    """Визуализирует границу решения"""
    h = 0.02
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu', edgecolors='black', s=40, alpha=0.7)
    ax.set_title(title, fontweight='bold', fontsize=11)
    ax.set_xlabel('Биомаркер 1')
    ax.set_ylabel('Биомаркер 2')

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

plot_decision_boundary(tree_clf, X_test_c, y_test_c, axes[0],
                      f'Одиночное дерево\nTest Acc = {tree_test_acc:.2%}')
plot_decision_boundary(rf_clf, X_test_c, y_test_c, axes[1],
                      f'Random Forest (100 деревьев)\nTest Acc = {rf_test_acc:.2%}')

axes[0].text(0.5, 0.02, 'Изломанная граница\n(высокий variance)', 
            transform=axes[0].transAxes, ha='center', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))
axes[1].text(0.5, 0.02, 'Гладкая граница\n(сниженный variance)', 
            transform=axes[1].transAxes, ha='center', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

plt.tight_layout()
plt.show()

## 4. Важность признаков (Feature Importance)

Одно из важных преимуществ Random Forest — возможность оценить важность каждого признака.

In [ ]:
# Генерируем данные с большим количеством признаков
X_features, y_features = make_classification(
    n_samples=500, n_features=10, n_informative=5, n_redundant=3, 
    n_repeated=0, n_classes=2, random_state=42
)

feature_names = [f'Ген {i+1}' for i in range(10)]

# Обучаем Random Forest
rf_importance = RandomForestClassifier(n_estimators=100, random_state=42)
rf_importance.fit(X_features, y_features)

# Важность признаков
importances = rf_importance.feature_importances_
indices = np.argsort(importances)[::-1]

# Визуализация
plt.figure(figsize=(10, 6))
plt.bar(range(10), importances[indices], color='steelblue', alpha=0.8)
plt.xticks(range(10), [feature_names[i] for i in indices], rotation=45)
plt.xlabel('Признаки (гены)', fontsize=12)
plt.ylabel('Важность', fontsize=12)
plt.title('Важность признаков для классификации\n(Feature Importance)', 
         fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nТоп-5 самых важных признаков:")
for i in range(5):
    print(f"{i+1}. {feature_names[indices[i]]}: {importances[indices[i]]:.3f}")

print("\n✓ Random Forest позволяет выявить важные биомаркеры!")

## 5. Out-of-Bag (OOB) Error

Random Forest использует bootstrap-выборки. Для каждого дерева ~37% данных остаются неиспользованными (out-of-bag). Их можно использовать для оценки ошибки без отдельной валидационной выборки!

In [ ]:
# Random Forest с OOB оценкой
rf_oob = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42)
rf_oob.fit(X_train_c, y_train_c)

oob_score = rf_oob.oob_score_
test_score = accuracy_score(y_test_c, rf_oob.predict(X_test_c))

print("\nOut-of-Bag (OOB) оценка:")
print(f"  OOB Accuracy:  {oob_score:.2%}")
print(f"  Test Accuracy: {test_score:.2%}")
print(f"\n✓ OOB score близок к тестовой точности!")
print("  Это позволяет оценить качество модели без отдельной валидации")

## 6. Кривые обучения (Learning Curves)

In [ ]:
# Сравниваем кривые обучения для дерева и Random Forest
train_sizes = np.linspace(0.1, 1.0, 10)

# Одиночное дерево
train_sizes_tree, train_scores_tree, val_scores_tree = learning_curve(
    DecisionTreeClassifier(max_depth=10, random_state=42),
    X_train_c, y_train_c, train_sizes=train_sizes, cv=5, n_jobs=-1
)

# Random Forest
train_sizes_rf, train_scores_rf, val_scores_rf = learning_curve(
    RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
    X_train_c, y_train_c, train_sizes=train_sizes, cv=5, n_jobs=-1
)

# График
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Одиночное дерево
axes[0].plot(train_sizes_tree, train_scores_tree.mean(axis=1), 'b-o', lw=2, label='Train')
axes[0].plot(train_sizes_tree, val_scores_tree.mean(axis=1), 'r-s', lw=2, label='Validation')
axes[0].fill_between(train_sizes_tree, 
                     val_scores_tree.mean(axis=1) - val_scores_tree.std(axis=1),
                     val_scores_tree.mean(axis=1) + val_scores_tree.std(axis=1),
                     alpha=0.2, color='r')
axes[0].set_xlabel('Размер выборки')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Learning Curve: Одиночное дерево\n(большой gap = высокий variance)', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0.6, 1.0])

# Random Forest
axes[1].plot(train_sizes_rf, train_scores_rf.mean(axis=1), 'b-o', lw=2, label='Train')
axes[1].plot(train_sizes_rf, val_scores_rf.mean(axis=1), 'r-s', lw=2, label='Validation')
axes[1].fill_between(train_sizes_rf, 
                     val_scores_rf.mean(axis=1) - val_scores_rf.std(axis=1),
                     val_scores_rf.mean(axis=1) + val_scores_rf.std(axis=1),
                     alpha=0.2, color='r')
axes[1].set_xlabel('Размер выборки')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Learning Curve: Random Forest\n(меньший gap = сниженный variance)', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0.6, 1.0])

plt.tight_layout()
plt.show()

print("\n✓ Random Forest показывает меньший разрыв между train и validation!")

## 7. Подбор гиперпараметров

In [ ]:
# Влияние основных гиперпараметров
n_trees_test = [10, 30, 50, 100, 200]
max_depths_test = [5, 10, 15, None]

results = []

for n_trees in n_trees_test:
    for max_depth in max_depths_test:
        rf = RandomForestClassifier(n_estimators=n_trees, max_depth=max_depth, random_state=42)
        scores = cross_val_score(rf, X_train_c, y_train_c, cv=5)
        results.append({
            'n_estimators': n_trees,
            'max_depth': str(max_depth) if max_depth else '∞',
            'cv_score': scores.mean(),
            'cv_std': scores.std()
        })

df_results = pd.DataFrame(results)
df_pivot = df_results.pivot(index='n_estimators', columns='max_depth', values='cv_score')

print("\nТочность (CV) для разных гиперпараметров:\n")
print(df_pivot.to_string())

# Визуализация
plt.figure(figsize=(10, 6))
for depth in max_depths_test:
    depth_str = str(depth) if depth else '∞'
    data = df_results[df_results['max_depth'] == depth_str]
    plt.plot(data['n_estimators'], data['cv_score'], 'o-', lw=2, markersize=8, label=f'max_depth={depth_str}')

plt.xlabel('Количество деревьев (n_estimators)', fontsize=12)
plt.ylabel('CV Accuracy', fontsize=12)
plt.title('Влияние гиперпараметров на качество Random Forest', fontsize=13, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

best_result = df_results.loc[df_results['cv_score'].idxmax()]
print(f"\n✓ Лучшая комбинация:")
print(f"  n_estimators = {best_result['n_estimators']:.0f}")
print(f"  max_depth = {best_result['max_depth']}")
print(f"  CV Accuracy = {best_result['cv_score']:.2%} ± {best_result['cv_std']:.2%}")

## 8. Резюме

### Bias-Variance Tradeoff для Random Forest:

| Параметр | Bias | Variance | Рекомендации |
|----------|------|----------|-------------|
| **n_estimators** ↑ | Не меняется | ↓ Уменьшается | Больше = лучше (до насыщения) |
| **max_depth** ↑ | ↓ Уменьшается | ↑ Увеличивается | Подбирать по CV |
| **max_features** ↓ | ↑ Увеличивается | ↓ Уменьшается | sqrt(n) для классификации, n/3 для регрессии |
| **min_samples_split** ↑ | ↑ Увеличивается | ↓ Уменьшается | 2-20 |

### Ключевые выводы:

1. **Random Forest снижает variance** по сравнению с одиночным деревом за счет усреднения
2. **Увеличение n_estimators почти никогда не приводит к overfitting**
3. **OOB error** позволяет оценить качество без отдельной валидации
4. **Feature importance** помогает в интерпретации и отборе признаков
5. **Random Forest устойчивее к шуму** и выбросам, чем одиночное дерево

### Сравнение с одиночным деревом:

| Аспект | Одиночное дерево | Random Forest |
|--------|-----------------|---------------|
| **Bias** | Зависит от глубины | Аналогично |
| **Variance** | Высокий ↑↑ | Низкий ↓ |
| **Интерпретируемость** | Высокая ✓✓ | Низкая ✗ |
| **Точность** | Средняя | Высокая ✓✓ |
| **Устойчивость** | Низкая | Высокая ✓ |
| **Скорость** | Быстро ✓✓ | Медленнее ✗ |

### Применение в биологии:

- **Анализ экспрессии генов**: выявление важных генов для заболеваний
- **Предсказание структуры белков**: классификация вторичных структур
- **Диагностика**: классификация заболеваний по множеству биомаркеров
- **Взаимодействия**: предсказание белок-белковых или белок-лигандных взаимодействий

### Когда использовать Random Forest:

✓ Табличные данные с множеством признаков
✓ Нужна высокая точность
✓ Важна оценка важности признаков
✓ Данные с шумом и выбросами
✓ Нелинейные зависимости

✗ Нужна полная интерпретируемость (используйте одиночное мелкое дерево)
✗ Критична скорость предсказания в реальном времени
✗ Очень большие датасеты (используйте XGBoost, LightGBM)

## 9. Задания для самостоятельной работы

1. **Влияние max_features**: Исследуйте, как параметр `max_features` влияет на корреляцию между деревьями и итоговую точность

2. **Экстремально случайные деревья (Extra Trees)**: Сравните `RandomForestClassifier` и `ExtraTreesClassifier` на том же датасете

3. **Реальные данные**: Загрузите датасет по экспрессии генов (например, breast cancer из sklearn) и постройте модель для классификации

4. **Partial Dependence Plots**: Используйте `sklearn.inspection.partial_dependence` для анализа влияния отдельных признаков

5. **Сравнение с градиентным бустингом**: Подготовьтесь к следующему ноутбуку — изучите различия между bagging и boosting